In [42]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
import matplotlib.pyplot as pyplot
import pickle
from matplotlib import style

style.use("ggplot")


In [43]:
import requests

BASE = "https://nbagopher-api.herokuapp.com/"
BASE_TEST = "http://localhost:5000/"
BBR_BASE = "https://basketball-reference.com"
SEASON = 2021
BBREF_SEASON = SEASON+1
NBA_API = f"http://data.nba.net/data/10s/prod/v1/{SEASON}/players.json"
player_data = requests.get(NBA_API)
player_data = player_data.json()['league']['standard']


In [44]:
x_data = []
y_data = []

for player in player_data:
    player_id = player['personId']
    player_stats = requests.get(BASE + f"player/compiled/{player_id}")
    player_stats = player_stats.json()
    try: 
        if (len(player_stats['player_basic_prev']) > 0 and int(player_stats['player_basic_latest']['games_played'])>=57):
            player_basic_latest = player_stats['player_basic_latest']
            player_basic_prev = player_stats['player_basic_prev']
            player_advanced_latest = player_stats['player_advanced_latest']
            player_advanced_prev = player_stats['player_advanced_prev']

            player_info = player_stats['player_info']
            x_data.append([int(player_info['years_pro'])-1, float(player_info['height_meters']), float(player_basic_prev['mpg']), float(player_basic_prev['ppg']), float(player_basic_prev['apg']), float(player_basic_prev['rpg']), float(player_advanced_prev['TS_pctg']), float(player_advanced_prev['BPM'])])
            y_data.append([float(player_basic_latest['mpg']), float(player_basic_latest['ppg']), float(player_basic_latest['apg']), float(player_basic_latest['rpg']), float(player_advanced_latest['TS_pctg']), float(player_advanced_latest['BPM'])])
    except:
        continue

In [45]:
best = 0
for i in range(500):
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x_data, y_data, test_size=0.2)
    model = linear_model.LinearRegression()
    model.fit(x_train, y_train)
    acc = model.score(x_test, y_test)
    print(acc)

    if acc > best:
        best = acc
        with open("future_perf.pickle", "wb") as f:
            pickle.dump(model, f)
print(best)


0.5950589258821007
0.5564603189854945
0.5951173704486413
0.5399683685104095
0.602307110035092
0.4452256530275444
0.5717311205654559
0.6142322016957599
0.5985289733023375
0.6198847371099672
0.6341316825514137
0.6635741469704948
0.42761526275138806
0.6087966814775571
0.633722957076624
0.5007971764856897
0.5830007324920082
0.6551499114591101
0.535082984553418
0.5509192946442456
0.553692743952726
0.7216268592496031
0.5244934540046592
0.5309096319146633
0.48253205296340423
0.7360484064965148
0.7374572879399697
0.7385173450400071
0.5938819006801467
0.5739839366877516
0.656586314433436
0.4732579528807594
0.5056615347980197
0.6929902003231837
0.6709346782479986
0.44695632704951094
0.6814829450923185
0.6953635755565474
0.6055252937817251
0.581993509687053
0.5635161310638677
0.557191666928036
0.6927093421849323
0.7263763827690356
0.6489347368731446
0.6533607731700488
0.5918624571294829
0.6281815377063767
0.6762259808145935
0.5951469122520398
0.5621988812848159
0.5821782285693846
0.61417997362049

In [39]:
pickle_in = open("future_perf.pickle", "rb")
model = pickle.load(pickle_in)

predictions = model.predict(x_test)

for i in range(len(predictions)):
    print(predictions[i], x_test[i], y_test[i])


[28.20329725 14.48666357  2.18923107  3.25879787  0.55344742 -1.42567583] [8, 1.98, 29.3, 15.6, 2.3, 3.4, 0.53, -2.3] [23.0, 10.0, 1.8, 2.6, 0.515, -3.7]
[33.90529533 19.80300663  6.13381057 11.5611623   0.56037465  4.31505984] [4, 2.11, 36.0, 20.3, 6.7, 12.0, 0.601, 3.8] [34.5, 18.9, 5.2, 12.1, 0.63, 3.6]
[22.75571051  9.19824064  2.15192605  4.13141257  0.58289536  0.03311204] [1, 1.96, 18.9, 7.0, 1.6, 3.6, 0.603, -0.2] [28.8, 10.8, 2.6, 5.3, 0.572, -0.7]
[30.75497957 15.87787614  4.86255848  3.88504975  0.53260312 -0.73687354] [1, 1.93, 31.2, 15.1, 4.8, 4.1, 0.54, -2.4] [27.3, 12.6, 2.9, 3.0, 0.561, -2.2]
[24.12773919 10.63850911  2.13447473  6.834372    0.58853208  0.14019842] [12, 2.03, 24.9, 12.2, 2.5, 7.4, 0.556, -0.1] [22.6, 13.4, 2.2, 7.2, 0.588, 4.1]
[20.98690477  7.5330431   1.58933503  3.54471327  0.54800809 -0.4014752 ] [0, 1.96, 17.0, 5.5, 0.9, 2.8, 0.524, -1.5] [27.2, 12.2, 1.9, 4.3, 0.538, -0.5]
[23.28398006 11.85357013  1.29334273  7.53620411  0.59263143  1.59765257] [